In [4]:
import os
import sys

from torch.utils.data import Dataset, DataLoader
import pandas as pd
import numpy as np
import SimpleITK as sitk
import nrrd
import vtk

import torch
from torch import nn
from torch.nn import functional as F
from torchvision import transforms

import pytorch_lightning as pl
import pickle
import monai 
import glob 
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D


sys.path.append('/mnt/raid/C1_ML_Analysis/source/famli-ultra-sim/')
sys.path.append('/mnt/raid/C1_ML_Analysis/source/famli-ultra-sim/dl')
import dl.loaders.ultrasound_dataset as ultrasound_dataset
import dl.transforms.ultrasound_transforms as ultrasound_transforms
import dl.nets.us_simulation_jit as us_simulation_jit
import dl.nets.us_simu as us_simu

import importlib

from pytorch3d.ops import (knn_points, 
                           knn_gather)

sys.path.append('/mnt/raid/C1_ML_Analysis/source/ShapeAXI/src/')
from shapeaxi.saxi_layers import SelfAttention
import math


In [ ]:
def positional_encoding(seq_len: int, d_model: int, tag: int, device='cpu') -> torch.Tensor:
    """
    Sinusoidal positional encoding with tag-based offset.

    Args:
        seq_len (int): Sequence length.
        d_model (int): Embedding dimension.
        tag (int): Unique tag for the sequence.
        device (str): Device to store the tensor.

    Returns:
        torch.Tensor: Positional encoding (seq_len, d_model).
    """
    pe = torch.zeros(seq_len, d_model, device=device)
    
    # Offset positions by a tag-dependent amount to make each sequence encoding unique
    position = torch.arange(tag * seq_len, (tag + 1) * seq_len, dtype=torch.float32, device=device).unsqueeze(1)

    div_term = torch.exp(torch.arange(0, d_model, 2, dtype=torch.float32, device=device) * (-math.log(10000.0) / d_model))
    
    pe[:, 0::2] = torch.sin(position * div_term)
    pe[:, 1::2] = torch.cos(position * div_term)
    
    return pe



In [8]:
penc = positional_encoding(seq_len=100, d_model=512, tag=9)

In [9]:
penc.shape

torch.Size([100, 512])